In [1]:
!pip install gymnasium
# مكتبة لاستيراد ملفات الشيفرة
!pip install nbimporter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 10.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# صنف استيراد ملفات الشيفرة
import nbformat

In [4]:
# مسار ملف الصنف
class_file_path='/content/drive/MyDrive/Classes/DroneEnvironment.ipynb'

In [5]:
# تحميل ملف شيفرة الصنف
with open(class_file_path) as f:
    notebook = nbformat.read(f, as_version=4)

In [6]:
# تنفيذ خلايا الشيفرة
for cell in notebook.cells:
    if cell.cell_type == "code":
        try:
            exec(cell.source)
        except Exception as e:
            print(f"Error executing cell: {e}")

In [7]:
# مثال
# الأبنية
buildings = [
    {'pos': [3, 3, 0], 'size': [5, 5, 5]},
    {'pos': [8, 8, 0], 'size': [1, 1, 1]}
]
# المناطق الممنوعة
prohibited_zones = [
    {'pos': [0, 2, 0], 'size': [2, 4, 4]},
    {'pos': [8, 1, 0], 'size': [2, 2, 5]}
]
# البداية
start_position = [1, 9, 0]
# الهدف
goal_position = [9, 0, 8]

# إنشاء البيئة
env = gym.make("Drone-v0", grid_size=10, start_position=start_position,buildings=buildings, prohibited_zones=prohibited_zones , max_episode_steps=200)

env.reset()
raw_env = env.unwrapped
raw_env.set_goal(goal_position )

# الإظهار
env.render()

In [8]:
# دالة لاختيار الفعل باستخدام إبسيلون
def epsilon_greedy(state, q_table, epsilon):
  # إذا كان العدد العشوائي أقل من إبسيلون، نستكشف
  if np.random.rand() < epsilon:
      # اختيار فعل عشوائي
      return env.action_space.sample()
  # إذا كان العدد العشوائي أكبر من إبسيلون، نستخدم المعرفة السابقة
  else:
      # اختيار الفعل الذي له أعلى قيمة
      return np.argmax(q_table[state])

In [9]:
# دالة التدريب
def train_agent(env):
    # تهبئة جدول القيم
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    # معاملات خوارزمية التعلم النوعي
    learning_rate = 0.1  # معدل التعلم
    discount_factor = 0.9  # عامل الخصم
    epsilon_start = 1.0  # قيمة إبسيلون للبداية
    epsilon_min = 0.01  # الحد الأدنى لقيمة إبسيلون
    epsilon_decay = 0.999995  # تقليل إبسيلون تدريجيًا مع مرور الحلقات
    # عدد الحلقات التدريبية
    num_episodes = 2000000
    epsilon=epsilon_start

      # حلقة التدريب الرئيسية
    for episode in range(num_episodes):
        # إعادة تهيئة البيئة في بداية كل حلقة
        state, info = env.reset()
        done = False
        # المكافأة الإجمالية للحلقة
        total_reward = 0

        # حلقة لتحديث القيم بناءً على الأفعال التي يتم اتخاذها
        while not done:
            # اختيار الفعل
            action = epsilon_greedy(state, q_table, epsilon)
            # تطبيق الفعل
            next_state, reward, terminated, truncated, info = env.step(action)

            # تحديث القيمة في الجدول
            q_table[state, action] = q_table[state, action] + learning_rate * (reward + discount_factor * np.max(q_table[next_state]) - q_table[state, action])
            # الانتقال للحالة التالية
            state = next_state
            # إضافة المكافأة إلى إجمالي مكافآت الحلقة
            total_reward += reward
            # التحقق مما إذا كانت الحلقة قد انتهت
            if terminated or truncated:
                done = True  # نهاية الحلقة
        # تقليل إبسيلون مع تقدم الحلقات
        epsilon = max(epsilon_min, epsilon * epsilon_decay )

        # طباعة التقدم كل 10000 حلقة لتتبع الأداء
        if (episode + 1) % 10000 == 0:
            print(f"Episode {episode+1}/{num_episodes}, Total Reward: {total_reward}, Epsilon: {epsilon:.4f}")
    return q_table

In [10]:
# تدريب الدرون
q_table = train_agent(env)

Episode 10000/200000, Total Reward: -4641, Epsilon: 0.9512
Episode 20000/200000, Total Reward: -4155, Epsilon: 0.9048
Episode 30000/200000, Total Reward: -1229, Epsilon: 0.8607
Episode 40000/200000, Total Reward: -4201, Epsilon: 0.8187
Episode 50000/200000, Total Reward: 644, Epsilon: 0.7788
Episode 60000/200000, Total Reward: -5244, Epsilon: 0.7408
Episode 70000/200000, Total Reward: -3217, Epsilon: 0.7047
Episode 80000/200000, Total Reward: -4453, Epsilon: 0.6703
Episode 90000/200000, Total Reward: -3021, Epsilon: 0.6376
Episode 100000/200000, Total Reward: 320, Epsilon: 0.6065
Episode 110000/200000, Total Reward: 996, Epsilon: 0.5769
Episode 120000/200000, Total Reward: -2921, Epsilon: 0.5488
Episode 130000/200000, Total Reward: -2425, Epsilon: 0.5220
Episode 140000/200000, Total Reward: 985, Epsilon: 0.4966
Episode 150000/200000, Total Reward: -2130, Epsilon: 0.4724
Episode 160000/200000, Total Reward: -3118, Epsilon: 0.4493
Episode 170000/200000, Total Reward: -1681, Epsilon: 0.42

In [11]:
import pickle
def save_q_table_and_env(q_table, env, q_table_file, env_file):
    # القيم النوعية
    with open(q_table_file, 'wb') as f:
        pickle.dump(q_table, f)
    # البيئة
    with open(env_file, 'wb') as f:
        pickle.dump(env, f)

In [12]:
# مسار مجلد العمل
working_folder='/content/drive/MyDrive/RLModels/'
# مسار ملف جدول القيم
q_table_Drone_path = working_folder + "q_table_Drone.pkl"
# مسار ملف البيئة
env_pkl_Drone_path = working_folder + "env_pkl_Drone.pkl"

In [13]:
save_q_table_and_env(q_table, env,q_table_Drone_path, env_pkl_Drone_path)